In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
from geopandas import GeoDataFrame
import xlrd
from random import randint
import requests
from io import BytesIO
from datetime import datetime

pd.set_option('display.max_columns', None)

## Reading spatial and excel files for 2025:

In [2]:
gdf = gpd.read_file("./shp/SF_DOE_PREC_2022_07_18_pg.shp", encoding='utf-8')
gdf = gdf.to_crs('EPSG:4269')

In [3]:
url = 'https://www.sfelections.org/results/20250916/data/20250925/sov.xlsx'

In [4]:
# date = datetime.today().strftime("%Y%m%d")

# election_date = "20250916"

# # Build the URL
# url = f"https://www.sfelections.org/results/{election_date}/data/{date}/sov.xlsx"

print(url)

https://www.sfelections.org/results/20250916/data/20250925/sov.xlsx


In [5]:
response = requests.get(url)

In [6]:
dfTurnout = pd.read_excel(BytesIO(response.content), sheet_name='Sheet1', header=4, skipfooter=10)
dfA = pd.read_excel(BytesIO(response.content), sheet_name='Sheet2', header=3, skipfooter=8)

In [7]:
dfA

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,Yes\n,Unnamed: 7,No\n,Unnamed: 9,Total Votes,Unnamed: 11
0,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,0.0,NaN,0.0,Election Day,53.0,65.43%,28.0,34.57%,81.0,NaN
4,Vote by Mail,2566.0,2.0,NaN,0.0,Vote by Mail,583.0,59.86%,391.0,40.14%,974.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Total,594.0,1.0,NaN,0.0,Total,252.0,80.00%,63.0,20.00%,315.0,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,0.0,NaN,0.0,Election Day,27.0,79.41%,7.0,20.59%,34.0,NaN
80,Vote by Mail,2383.0,2.0,NaN,0.0,Vote by Mail,737.0,75.13%,244.0,24.87%,981.0,NaN


In [8]:
dfTurnout

,Precinct,Registered\nVoters,Unnamed: 2,Voters Cast,Unnamed: 4,% Turnout,Unnamed: 6,Unnamed: 7
0,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,NaN,81.0,NaN,0.031567,NaN,NaN
4,Vote by Mail,2566.0,NaN,976.0,NaN,0.380359,NaN,NaN
...,...,...,...,...,...,...,...,...
77,Total,594.0,NaN,316.0,NaN,0.531987,NaN,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,NaN,34.0,NaN,0.014268,NaN,NaN
80,Vote by Mail,2383.0,NaN,983.0,NaN,0.412505,NaN,NaN


### Formatting

In [9]:
###
### TURNOUT
###

#trim and rename columns
dfTurnout = dfTurnout[['Precinct','Registered\nVoters','Voters Cast']]
dfTurnout.columns = ['precinct','registered_voters','votes_cast']

# shift precinct column

dfTurnout['precinct'] = dfTurnout['precinct'].shift(3)

#drop unnecessary ones
dfTurnout = dfTurnout.dropna()
dfTurnout = dfTurnout[dfTurnout["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfTurnout['precinct'] = dfTurnout['precinct'].str.replace('PCT ','').str.replace(' MB','')

#change data types
dfTurnout.registered_voters = dfTurnout.registered_voters.astype(int)
dfTurnout.votes_cast = dfTurnout.votes_cast.astype(int)

#create turnout column
dfTurnout['turnout'] = round((dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100, 1)
dfTurnout['turnout'] = dfTurnout['turnout'].fillna(0)

/tmp/ipykernel_2158/2972828771.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTurnout['precinct'] = dfTurnout['precinct'].shift(3)


In [10]:
dfTurnout

,precinct,registered_voters,votes_cast,turnout
5,9401,2566,1057,41.2
9,9403,2247,971,43.2
13,9404,3243,1364,42.1
17,9406,2415,832,34.5
21,9408,2979,1300,43.6
25,9413,3296,1401,42.5
29,9414,2227,873,39.2
33,9417,2211,1020,46.1
37,9421,2106,781,37.1
41,9423,2363,1145,48.5


In [11]:
###
### PROPS
###

def process_proposition(df):
    # Standardize column names
    df.columns = df.columns.str.replace('\n', '', regex=True)
    df.columns = df.columns.str.replace('"', '', regex=True)
    df.columns = df.columns.str.replace('.', '', regex=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_', regex=True)
    df.columns = df.columns.str.replace('_$', '', regex=True)

    # if the column name contains "bonds_-_yes", replace it with "yes"

    df.columns = df.columns.str.replace('bonds_-_yes', 'yes', regex=True)
    df.columns = df.columns.str.replace('bonds_-_no', 'no', regex=True)
    
    # Trim and rename columns to focus on necessary data
    df = df[['precinct', 'registered_voters', 'yes', 'no', 'total_votes']]
    df.columns = ['precinct', 'registered_voters', 'yes', 'no', 'votes_cast']

    # shift the precinct column up by 3 rows

    df['precinct'] = df['precinct'].shift(3)

    # Drop rows where precinct is NaN after the shift
    df = df.dropna(subset=['precinct'])
    df = df[df['precinct'].str.contains('PCT', na=False)]

    # Clean up precinct column data
    df['precinct'] = df['precinct'].str.replace('PCT ', '', regex=True).str.replace(' MB', '', regex=True)

    # Ensure numeric columns are of the correct type
    # df['registered_voters'] = pd.to_numeric(df['registered_voters'], errors='coerce').fillna(0).astype(int)
    # df['votes_cast'] = pd.to_numeric(df['votes_cast'], errors='coerce').fillna(0).astype(int)

    # Calculate turnout
    df['turnout'] = round((df['votes_cast'] / df['registered_voters']) * 100, 1).fillna(0)

    return df

In [12]:
dfA = process_proposition(dfA)

KeyError: "None of [Index(['precinct', 'registered_voters', 'yes', 'no', 'total_votes'], dtype='object')] are in the [columns]"

In [13]:
bins = [0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 100]
labels = ['Less than 25%', '25-30%', '30-35%', '35-40%', '40-45%', '45-50%', '50-55%', '55-60%', '60-65%', '65-70%', '70-75%', '75% and more']

In [14]:
# I want to generate random numbers for yes and no columns in dfA, but the sum of yes and no should be  no more than registered_voters

# def generate_yes_no(row):
#     registered_voters = row['registered_voters']
#     if registered_voters == 0:
#         return pd.Series([0, 0])
    
#     yes = randint(0, registered_voters)
#     no = randint(0, registered_voters - yes)
    
#     return pd.Series([yes, no])

# # make votes_cast the sum of yes and no

# dfA[['yes', 'no']] = dfA.apply(generate_yes_no, axis=1)
# dfA['votes_cast'] = dfA['yes'] + dfA['no']
# dfA['turnout'] = round((dfA['votes_cast'] / dfA['registered_voters']) * 100, 1).fillna(0)

In [15]:
dfTurnout.head()

,precinct,registered_voters,votes_cast,turnout
5,9401,2566,1057,41.2
9,9403,2247,971,43.2
13,9404,3243,1364,42.1
17,9406,2415,832,34.5
21,9408,2979,1300,43.6


In [16]:
# do the same thing for dfTurnout. This time we are just populating the votes_cast column with random numbers

# dfTurnout['votes_cast'] = dfTurnout.apply(
#     lambda row: randint(0, row['registered_voters']) if row['registered_voters'] > 0 else 0,
#     axis=1
# )
# dfTurnout['turnout'] = round((dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100, 1).fillna(0)

In [17]:
dfA

,,,,,,,,,,,,
0,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,0.0,NaN,0.0,Election Day,53.0,65.43%,28.0,34.57%,81.0,NaN
4,Vote by Mail,2566.0,2.0,NaN,0.0,Vote by Mail,583.0,59.86%,391.0,40.14%,974.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Total,594.0,1.0,NaN,0.0,Total,252.0,80.00%,63.0,20.00%,315.0,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,0.0,NaN,0.0,Election Day,27.0,79.41%,7.0,20.59%,34.0,NaN
80,Vote by Mail,2383.0,2.0,NaN,0.0,Vote by Mail,737.0,75.13%,244.0,24.87%,981.0,NaN


In [18]:
# Calculate voter turnout percentage
dfTurnout['yes_perc'] = (dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100

# Categorize turnout percentages into bins
dfTurnout['yes_perc'] = pd.cut(dfTurnout['yes_perc'], bins=bins, labels=labels, include_lowest=True)

In [19]:
dfA['precinct'] = dfA['precinct'].astype(int)
dfA['yes_perc'] = (dfA['yes'] / dfA['votes_cast']) * 100
dfA['yes_perc_bin'] = pd.cut(dfA['yes_perc'], bins=bins, labels=labels, include_lowest=True)
dfA['yes_perc'] = dfA['yes_perc'].fillna('no data')
dfA['yes_perc_bin'] = dfA['yes_perc_bin'].astype(str)

KeyError: 'precinct'

## Bar chart

copy for bar chart

In [20]:
# yes,no,margin,threshold,total,Proposition,yes_p,no_p
# 282968.0,94329.0,188639.0,50.0,377297.0,A,75.0,25.0

In [21]:
# make a bottom row with the totals of each column

total_row = pd.DataFrame({
    'precinct': ['Total'],
    'registered_voters': [dfA['registered_voters'].sum()],
    'yes': [dfA['yes'].sum()],
    'no': [dfA['no'].sum()],
    'votes_cast': [dfA['votes_cast'].sum()]
    })

total_row

KeyError: 'registered_voters'

In [22]:
# make a yes_p and no_p column that is the percentage of yes and no votes

total_row['yes_p'] = round((total_row['yes'] / total_row['votes_cast']) * 100, 1).fillna(0)
total_row['no_p'] = round((total_row['no'] / total_row['votes_cast']) * 100, 1).fillna(0)

NameError: name 'total_row' is not defined

In [23]:
# make a column that is called proposition and set it to A

total_row['proposition'] = 'A'
total_row['threshold'] = 50.0

NameError: name 'total_row' is not defined

In [24]:
total_row

NameError: name 'total_row' is not defined

In [25]:
# add a turnout column

total_row['turnout'] = round((total_row['votes_cast'] / total_row['registered_voters']) * 100, 1).fillna(0)

NameError: name 'total_row' is not defined

In [26]:
# export to docs/bar-chart/data.csv

total_row.to_csv('./docs/bar-chart/data.csv', index=False)

NameError: name 'total_row' is not defined

## Reading files for 2022

In [27]:
url = 'https://www.sfelections.org/results/20221108/data/20221201/sov.xlsx'

In [28]:
response = requests.get(url)

In [29]:
dfd4 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet43', header=3, skipfooter=8)

In [30]:
dfd4.head()

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,JOEL ENGARDIO\n,Unnamed: 7,GORDON MAR\n,Unnamed: 9,Write-in\n,Unnamed: 11,Unnamed: 12,Total Votes
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,1215.0,9.0,NaN,0.0,Election Day,44.0,41.90%,61.0,58.10%,0.0,NaN,0.00%,105.0
4,Vote by Mail,1215.0,56.0,NaN,0.0,Vote by Mail,223.0,38.65%,354.0,61.35%,0.0,NaN,0.00%,577.0


In [31]:
### DISTRICT 4

# make all columns lowercase and remove spaces
dfd4.columns = dfd4.columns.str.replace('\n', '')
dfd4.columns = dfd4.columns.str.lower().str.replace(' ', '_')
dfd4.columns = dfd4.columns.str.replace('_$', '')

dfd4['precinct'] = dfd4['precinct'].shift(3)

dfd4 = dfd4[['precinct', 'registered_voters', 'joel_engardio', 'gordon_mar','total_votes']]

#drop unnecessary ones
dfd4 = dfd4.dropna()
dfd4 = dfd4[dfd4["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd4['precinct'] = dfd4['precinct'].str.replace('PCT ','').str.replace(' MB','')

KeyError: "['joel_engardio', 'gordon_mar'] not in index"

In [32]:
groupings = {
    "9401": ["9401", "9402"], 
    "9403": ["9403", "9411"],
    "9404": ["9404", "9405", "9412"],
    "9406": ["9406", "9407"],
    "9408": ["9408", "9409", "9416"],
    "9413": ["9413", "9418", "9419"], 
    "9414": ["9414", "9415"],
    "9417": ["9417", "9424"],
    "9421": ["9421", "9422"], 
    "9423": ["9423", "9428"], 
    "9425": ["9425", "9426", "9431"],
    "9427": ["9427", "9433", "9434"],
    "9429": ["9429", "9436", "9438"], 
    "9432": ["9432", "9441"], 
    "9435": ["9435", "9443"],
    "9437": ["9437", "9444", "9445"],
    "9439": ["9439","9446"], 
    "9442": ["9442", "9447", "9448"],
    "9449": ["9449"],
    "9451": ["9451", "9452"]
}

In [33]:
dfd4

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,joel_engardio_,unnamed:_7,gordon_mar_,unnamed:_9,write-in_,unnamed:_11,unnamed:_12,total_votes
0,None,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Countywide,1215.0,9.0,NaN,0.0,Election Day,44.0,41.90%,61.0,58.10%,0.0,NaN,0.00%,105.0
4,Electionwide,1215.0,56.0,NaN,0.0,Vote by Mail,223.0,38.65%,354.0,61.35%,0.0,NaN,0.00%,577.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,PCT 9451,1168.0,96.0,NaN,1.0,Total,489.0,69.66%,213.0,30.34%,0.0,NaN,0.00%,702.0
186,Election Day,NaN,NaN,NaN,NaN,PCT 9452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,Vote by Mail,1052.0,8.0,NaN,0.0,Election Day,32.0,72.73%,12.0,27.27%,0.0,NaN,0.00%,44.0
188,Total,1052.0,58.0,NaN,0.0,Vote by Mail,361.0,66.61%,181.0,33.39%,0.0,NaN,0.00%,542.0


In [34]:
# If a precint matches a key in the groupings dictionary, add up the rows that match the values in the list and create a new row with the key as the precinct

def group_precincts(df, groupings):
    grouped_data = []

    for key, values in groupings.items():
        subset = df[df['precinct'].isin(values)]
        if not subset.empty:
            aggregated_row = {
                'precinct': key,
                'registered_voters': subset['registered_voters'].sum(),
                'joel_engardio': subset['joel_engardio'].sum(),
                'gordon_mar': subset['gordon_mar'].sum(),
                'total_votes': subset['total_votes'].sum()
            }
            grouped_data.append(aggregated_row)

    grouped_df = pd.DataFrame(grouped_data)
    return grouped_df

dfd4_grouped = group_precincts(dfd4, groupings)

In [35]:
dfd4_grouped

""


In [36]:
dfd4_grouped["joel_engardio_p"] = round((dfd4_grouped["joel_engardio"] / dfd4_grouped["total_votes"]) * 100, 1)
dfd4_grouped["gordon_mar_p"] = round((dfd4_grouped["gordon_mar"] / dfd4_grouped["total_votes"]) * 100, 1)

KeyError: 'joel_engardio'

In [37]:
dfd4_grouped

""


In [38]:
# make a tunrout column

dfd4_grouped['turnout'] = round((dfd4_grouped['total_votes'] / dfd4_grouped['registered_voters']) * 100, 1).fillna(0)

KeyError: 'total_votes'

In [39]:
dfd4_grouped['winner'] = dfd4_grouped[['joel_engardio_p', 'gordon_mar_p']].idxmax(axis=1).str.replace('_p', '')

KeyError: "None of [Index(['joel_engardio_p', 'gordon_mar_p'], dtype='object')] are in the [columns]"

# Reading files for 2024

In [40]:
url = 'https://www.sfelections.org/results/20241105/data/20241203/sov.xlsx'

In [41]:
response = requests.get(url)

In [42]:
dfK = pd.read_excel(BytesIO(response.content), sheet_name='Sheet50', header=3, skipfooter=6)

In [43]:
dfK = process_proposition(dfK)

KeyError: "None of [Index(['precinct', 'registered_voters', 'yes', 'no', 'total_votes'], dtype='object')] are in the [columns]"

In [44]:
dfK

,,,,,,,,,,,,
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 1101,NaN,NaN,NaN,NaN,PCT 1101,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,881.0,20.0,NaN,0.0,Election Day,31.0,35.63%,56.0,64.37%,87.0,NaN
4,Vote by Mail,881.0,35.0,NaN,0.0,Vote by Mail,201.0,38.95%,315.0,61.05%,516.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2055,Election Day,0.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
2056,Vote by Mail,0.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN
2057,Total,0.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN
2058,Electionwide - Total,522265.0,32912.0,NaN,114.0,Electionwide - Total,206042.0,54.73%,170447.0,45.27%,376489.0,NaN


In [45]:
dfK.registered_voters.sum()

AttributeError: 'DataFrame' object has no attribute 'registered_voters'

In [46]:
dfK.votes_cast.sum()

AttributeError: 'DataFrame' object has no attribute 'votes_cast'

In [47]:
dfK

,,,,,,,,,,,,
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 1101,NaN,NaN,NaN,NaN,PCT 1101,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,881.0,20.0,NaN,0.0,Election Day,31.0,35.63%,56.0,64.37%,87.0,NaN
4,Vote by Mail,881.0,35.0,NaN,0.0,Vote by Mail,201.0,38.95%,315.0,61.05%,516.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2055,Election Day,0.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
2056,Vote by Mail,0.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN
2057,Total,0.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN
2058,Electionwide - Total,522265.0,32912.0,NaN,114.0,Electionwide - Total,206042.0,54.73%,170447.0,45.27%,376489.0,NaN


In [48]:
dfK.votes_cast.sum() / dfK.registered_voters.sum() * 100

AttributeError: 'DataFrame' object has no attribute 'votes_cast'

In [49]:
dfK.yes.sum()

AttributeError: 'DataFrame' object has no attribute 'yes'

In [50]:
dfK.yes.sum(0)

AttributeError: 'DataFrame' object has no attribute 'yes'

In [51]:
dfK.no.sum()

AttributeError: 'DataFrame' object has no attribute 'no'

In [52]:
dfd4.total_votes.sum() / dfd4.registered_voters.sum() * 100

np.float64(37.20175566326212)

In [53]:
dfd4.total_votes.sum()

np.float64(53652.0)

In [54]:
dfd4

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,joel_engardio_,unnamed:_7,gordon_mar_,unnamed:_9,write-in_,unnamed:_11,unnamed:_12,total_votes
0,None,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Countywide,1215.0,9.0,NaN,0.0,Election Day,44.0,41.90%,61.0,58.10%,0.0,NaN,0.00%,105.0
4,Electionwide,1215.0,56.0,NaN,0.0,Vote by Mail,223.0,38.65%,354.0,61.35%,0.0,NaN,0.00%,577.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,PCT 9451,1168.0,96.0,NaN,1.0,Total,489.0,69.66%,213.0,30.34%,0.0,NaN,0.00%,702.0
186,Election Day,NaN,NaN,NaN,NaN,PCT 9452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,Vote by Mail,1052.0,8.0,NaN,0.0,Election Day,32.0,72.73%,12.0,27.27%,0.0,NaN,0.00%,44.0
188,Total,1052.0,58.0,NaN,0.0,Vote by Mail,361.0,66.61%,181.0,33.39%,0.0,NaN,0.00%,542.0


In [55]:
def group_precincts_k(df, groupings):
    grouped_data = []

    for key, values in groupings.items():
        subset = df[df['precinct'].isin(values)]
        if not subset.empty:
            aggregated_row = {
                'precinct': key,
                'registered_voters': subset['registered_voters'].sum(),
                'yes': subset['yes'].sum(),
                'no': subset['no'].sum(),
                'registered_voters': subset['registered_voters'].sum(),
                'votes_cast': subset['votes_cast'].sum()
            }
            grouped_data.append(aggregated_row)

    grouped_df = pd.DataFrame(grouped_data)
    return grouped_df

In [56]:
dfK = group_precincts_k(dfK, groupings)

KeyError: 'precinct'

In [57]:
# I only want to keep the precincts that are keys in the groupings dictionary

dfK = dfK[dfK['precinct'].isin(groupings.keys())]

KeyError: 'precinct'

In [58]:
dfK.votes_cast.sum() / dfK.registered_voters.sum() * 100

AttributeError: 'DataFrame' object has no attribute 'votes_cast'

In [59]:
# make a turnout column

dfK['turnout'] = round((dfK['votes_cast'] / dfK['registered_voters']) * 100, 1).fillna(0)

KeyError: 'votes_cast'

In [60]:
dfK['precinct'] = dfK['precinct'].astype(int)
dfK['yes_perc'] = (dfK['yes'] / dfK['votes_cast']) * 100
dfK['yes_perc_bin'] = pd.cut(dfK['yes_perc'], bins=bins, labels=labels, include_lowest=True)
dfK['yes_perc'] = dfK['yes_perc'].fillna('no data')
dfK['yes_perc_bin'] = dfK['yes_perc_bin'].astype(str)

KeyError: 'precinct'

In [61]:
dfK

,,,,,,,,,,,,
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 1101,NaN,NaN,NaN,NaN,PCT 1101,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,881.0,20.0,NaN,0.0,Election Day,31.0,35.63%,56.0,64.37%,87.0,NaN
4,Vote by Mail,881.0,35.0,NaN,0.0,Vote by Mail,201.0,38.95%,315.0,61.05%,516.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2055,Election Day,0.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
2056,Vote by Mail,0.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN
2057,Total,0.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN
2058,Electionwide - Total,522265.0,32912.0,NaN,114.0,Electionwide - Total,206042.0,54.73%,170447.0,45.27%,376489.0,NaN


# Prepping gdf

In [62]:
gdf = gdf[['Prec_2022','geometry']]
gdf.Prec_2022 = gdf.Prec_2022.astype(str)

In [63]:
gdf

,Prec_2022,geometry
0,7042,"POLYGON ((-122.42165 37.71029, -122.421 37.710..."
1,1107,"POLYGON ((-122.45595 37.71134, -122.45593 37.7..."
2,1145,"POLYGON ((-122.44617 37.71104, -122.4455 37.70..."
3,7043,"POLYGON ((-122.41508 37.71166, -122.41289 37.7..."
4,7046,"POLYGON ((-122.40973 37.71195, -122.40883 37.7..."
...,...,...
509,9201,"POLYGON ((-122.47754 37.81011, -122.4775 37.80..."
510,9202,"POLYGON ((-122.47725 37.81102, -122.47711 37.8..."
511,7301,"POLYGON ((-122.41991 37.81163, -122.4199 37.81..."
512,7645,"POLYGON ((-122.36862 37.83116, -122.36733 37.8..."


In [64]:
groupings = {
    "9401": ["9401", "9402"], 
    "9403": ["9403", "9411"],
    "9404": ["9404", "9405", "9412"],
    "9406": ["9406", "9407"],
    "9408": ["9408", "9409", "9416"],
    "9413": ["9413", "9418", "9419"], 
    "9414": ["9414", "9415"],
    "9417": ["9417", "9424"],
    "9421": ["9421", "9422"], 
    "9423": ["9423", "9428"], 
    "9425": ["9425", "9426", "9431"],
    "9427": ["9427", "9433", "9434"],
    "9429": ["9429", "9436", "9438"], 
    "9432": ["9432", "9441"], 
    "9435": ["9435", "9443"],
    "9437": ["9437", "9444", "9445"],
    "9439": ["9439","9446"], 
    "9442": ["9442", "9447", "9448"],
    "9449": ["9449"],
    "9451": ["9451", "9452"]
}

In [65]:
# I want to merge the precincts in gdf based on the groupings above. The new precinct column should be the key of the groupings dict

gdf['precinct'] = gdf['Prec_2022'].astype(str)

gdf['precinct'] = gdf['precinct'].apply(lambda x: next((key for key, values in groupings.items() if x in values), x))

gdf = gdf.dissolve(by='precinct', as_index=False).reset_index(drop=True)

gdf

,precinct,geometry,Prec_2022
0,1101,"POLYGON ((-122.47167 37.72163, -122.47078 37.7...",1101
1,1102,"POLYGON ((-122.46802 37.7161, -122.468 37.7142...",1102
2,1103,"POLYGON ((-122.46625 37.71968, -122.46715 37.7...",1103
3,1104,"POLYGON ((-122.46266 37.72, -122.46267 37.7199...",1104
4,1105,"POLYGON ((-122.46263 37.71742, -122.46262 37.7...",1105
...,...,...,...
482,9808,"POLYGON ((-122.44106 37.73497, -122.44031 37.7...",9808
483,9901,"POLYGON ((-122.43251 37.73311, -122.43233 37.7...",9901
484,9902,"POLYGON ((-122.40396 37.73161, -122.4034 37.73...",9902
485,9903,"POLYGON ((-122.33081 37.78579, -122.33061 37.7...",9903


In [66]:
# export to geojson

gdf.to_file("sf_precincts_merged.geojson", driver='GeoJSON', encoding='utf-8')

# Merging

In [67]:
# make dfTurnout yes_perc a string too

dfTurnout['yes_perc'] = dfTurnout['yes_perc'].astype(str)

In [68]:
dfTurnout

,precinct,registered_voters,votes_cast,turnout,yes_perc
5,9401,2566,1057,41.2,40-45%
9,9403,2247,971,43.2,40-45%
13,9404,3243,1364,42.1,40-45%
17,9406,2415,832,34.5,30-35%
21,9408,2979,1300,43.6,40-45%
25,9413,3296,1401,42.5,40-45%
29,9414,2227,873,39.2,35-40%
33,9417,2211,1020,46.1,45-50%
37,9421,2106,781,37.1,35-40%
41,9423,2363,1145,48.5,45-50%


In [69]:
gdf

,precinct,geometry,Prec_2022
0,1101,"POLYGON ((-122.47167 37.72163, -122.47078 37.7...",1101
1,1102,"POLYGON ((-122.46802 37.7161, -122.468 37.7142...",1102
2,1103,"POLYGON ((-122.46625 37.71968, -122.46715 37.7...",1103
3,1104,"POLYGON ((-122.46266 37.72, -122.46267 37.7199...",1104
4,1105,"POLYGON ((-122.46263 37.71742, -122.46262 37.7...",1105
...,...,...,...
482,9808,"POLYGON ((-122.44106 37.73497, -122.44031 37.7...",9808
483,9901,"POLYGON ((-122.43251 37.73311, -122.43233 37.7...",9901
484,9902,"POLYGON ((-122.40396 37.73161, -122.4034 37.73...",9902
485,9903,"POLYGON ((-122.33081 37.78579, -122.33061 37.7...",9903


In [70]:
dfTurnout = gdf.merge(dfTurnout, right_on='precinct', left_on='precinct')
dfTurnout = dfTurnout[['precinct','registered_voters','votes_cast','yes_perc','turnout','geometry']]
gdfTurnout = GeoDataFrame(dfTurnout, crs="EPSG:4269", geometry='geometry')

In [71]:
dfA.dtypes

     object
    float64
    float64
    float64
    float64
     object
    float64
     object
    float64
     object
    float64
    float64
dtype: object

In [72]:
# make dfA precinct string

dfA['precinct'] = dfA['precinct'].astype(str)

KeyError: 'precinct'

In [73]:
# merge 

dfA = gdf.merge(dfA, on='precinct')

KeyError: 'precinct'

In [74]:
dfA.columns

Index(['', '', '', '', '', '', '', '', '', '', '', ''], dtype='object')

In [75]:
dfA

,,,,,,,,,,,,
0,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,0.0,NaN,0.0,Election Day,53.0,65.43%,28.0,34.57%,81.0,NaN
4,Vote by Mail,2566.0,2.0,NaN,0.0,Vote by Mail,583.0,59.86%,391.0,40.14%,974.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Total,594.0,1.0,NaN,0.0,Total,252.0,80.00%,63.0,20.00%,315.0,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,0.0,NaN,0.0,Election Day,27.0,79.41%,7.0,20.59%,34.0,NaN
80,Vote by Mail,2383.0,2.0,NaN,0.0,Vote by Mail,737.0,75.13%,244.0,24.87%,981.0,NaN


In [76]:
dfA = dfA[['precinct','registered_voters','yes','no','votes_cast','turnout','yes_perc','yes_perc_bin','geometry']]
gdfA = GeoDataFrame(dfA, crs="EPSG:4269", geometry='geometry')

KeyError: "None of [Index(['precinct', 'registered_voters', 'yes', 'no', 'votes_cast', 'turnout',\n       'yes_perc', 'yes_perc_bin', 'geometry'],\n      dtype='object')] are in the [columns]"

In [77]:
# merge gdf dfd4_grouped

dfd4_grouped['precinct'] = dfd4_grouped['precinct'].astype(str)

gdf4 = gdf.merge(dfd4_grouped, on='precinct')

gdf4


KeyError: 'precinct'

In [78]:
gdf

,precinct,geometry,Prec_2022
0,1101,"POLYGON ((-122.47167 37.72163, -122.47078 37.7...",1101
1,1102,"POLYGON ((-122.46802 37.7161, -122.468 37.7142...",1102
2,1103,"POLYGON ((-122.46625 37.71968, -122.46715 37.7...",1103
3,1104,"POLYGON ((-122.46266 37.72, -122.46267 37.7199...",1104
4,1105,"POLYGON ((-122.46263 37.71742, -122.46262 37.7...",1105
...,...,...,...
482,9808,"POLYGON ((-122.44106 37.73497, -122.44031 37.7...",9808
483,9901,"POLYGON ((-122.43251 37.73311, -122.43233 37.7...",9901
484,9902,"POLYGON ((-122.40396 37.73161, -122.4034 37.73...",9902
485,9903,"POLYGON ((-122.33081 37.78579, -122.33061 37.7...",9903


In [79]:
# merge the gdf with dfK

dfK['precinct'] = dfK['precinct'].astype(str)

gdfK = gdf.merge(dfK, on='precinct')

KeyError: 'precinct'

In [80]:
# export gdfK to geojson

gdfK.to_file("./docs/propK/propK.geojson", driver='GeoJSON', encoding='utf-8')
gdfK.to_file("./docs/comparison/propK.geojson", driver='GeoJSON', encoding='utf-8')

NameError: name 'gdfK' is not defined

In [81]:
gdfTurnout

,precinct,registered_voters,votes_cast,yes_perc,turnout,geometry
0,9401,2566,1057,40-45%,41.2,"POLYGON ((-122.50593 37.76042, -122.507 37.760..."
1,9403,2247,971,40-45%,43.2,"POLYGON ((-122.49555 37.75714, -122.49602 37.7..."
2,9404,3243,1364,40-45%,42.1,"POLYGON ((-122.48959 37.7574, -122.49066 37.75..."
3,9406,2415,832,30-35%,34.5,"POLYGON ((-122.47712 37.76356, -122.47697 37.7..."
4,9408,2979,1300,40-45%,43.6,"POLYGON ((-122.50112 37.75315, -122.50219 37.7..."
5,9413,3296,1401,40-45%,42.5,"POLYGON ((-122.48492 37.752, -122.48599 37.751..."
6,9414,2227,873,35-40%,39.2,"POLYGON ((-122.4766 37.75611, -122.47713 37.75..."
7,9417,2211,1020,45-50%,46.1,"POLYGON ((-122.49518 37.75165, -122.49516 37.7..."
8,9421,2106,781,35-40%,37.1,"POLYGON ((-122.47635 37.75238, -122.47742 37.7..."
9,9423,2363,1145,45-50%,48.5,"POLYGON ((-122.50407 37.74928, -122.50394 37.7..."


In [82]:
# export both 

gdfTurnout.to_file("./docs/turnout/turnout.geojson", driver='GeoJSON', encoding='utf-8')
gdfA.to_file("./docs/propA/propA.geojson", driver='GeoJSON', encoding='utf-8')
gdfA.to_file("./docs/comparison/propA.geojson", driver='GeoJSON', encoding='utf-8')
gdfA.to_file("./docs/comparison-2022/propA.geojson", driver='GeoJSON', encoding='utf-8')

NameError: name 'gdfA' is not defined

In [83]:
# export gdf4

gdf4.to_file("./docs/2022/d4.geojson", driver='GeoJSON', encoding='utf-8')
gdf4.to_file("./docs/comparison/d4.geojson", driver='GeoJSON', encoding='utf-8')
gdf4.to_file("./docs/comparison-2022/d4.geojson", driver='GeoJSON', encoding='utf-8')

NameError: name 'gdf4' is not defined